In [ ]:
# ==========================================
# 1. Mount Google Drive
# ==========================================
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==========================================
# 2. Imports & Paths
# ==========================================
import os
import shutil
import re

S1_VVVH_DIR = "/content/drive/MyDrive/S1_VVVH"
OUTPUT_DIR = "/content/drive/MyDrive/S1_VVVH_Filtered"

FIX_LIST_FILE = "/content/drive/MyDrive/S1_VVVH/S1_filename.txt"

# Create output folder if not exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# ==========================================
# 3. Load fixed S1 filenames
# ==========================================
with open(FIX_LIST_FILE, "r") as f:
    fixed_files = set(line.strip() for line in f if line.strip())

print(f"Loaded fixed S1 filenames: {len(fixed_files)}")

Loaded fixed S1 filenames: 74


In [ ]:
# ==========================================
# 4. Regex for ID handling
# ==========================================
# Matches:
# 1_...
# 1A_...
pattern_id = re.compile(r'^(\d+)(A?)_(.+)$')

In [ ]:
# ==========================================
# 5. Filter & Rename
# ==========================================
copied_valid = 0
copied_renamed = 0
skipped = []

for file in os.listdir(S1_VVVH_DIR):

    if not file.endswith(".tif"):
        continue

    src_path = os.path.join(S1_VVVH_DIR, file)

    # Case 1: Valid file → copy as-is
    if file in fixed_files:
        dst_path = os.path.join(OUTPUT_DIR, file)
        shutil.copy2(src_path, dst_path)
        copied_valid += 1
        continue

    # Case 2: Unused file → rename with X
    match = pattern_id.match(file)

    if not match:
        skipped.append(file)
        continue

    id_num = match.group(1)
    asc_flag = match.group(2)  # A or ''
    rest = match.group(3)

    new_prefix = f"{id_num}{asc_flag}X"
    new_name = f"{new_prefix}_{rest}"

    dst_path = os.path.join(OUTPUT_DIR, new_name)
    shutil.copy2(src_path, dst_path)
    copied_renamed += 1

In [ ]:
# ==========================================
# 6. Summary
# ==========================================
print("\nFILTERING COMPLETE\n")
print(f"Valid files copied (unchanged): {copied_valid}")
print(f"Unused files renamed + copied : {copied_renamed}")
print(f"Skipped (pattern mismatch)    : {len(skipped)}")

if skipped:
    print("\nSkipped files:")
    for s in skipped[:10]:
        print(" -", s)


FILTERING COMPLETE

Valid files copied (unchanged): 74
Unused files renamed + copied : 90
Skipped (pattern mismatch)    : 0
